## Spark_DataFrame_Joins

| 비교 항목           | Pandas (`pd.merge`)                      | Spark (`df.join`)                        |
| :-------------- | :--------------------------------------- | :--------------------------------------- |
| **문법 스타일**      | `pd.merge(left, right)`<br>(독립 함수 형태 선호) | `left.join(right)`<br>(메서드 체이닝 형태)       |
| **컬럼명 중복 시**    | **자동 해결** (`_x`, `_y` 접미사 붙여줌)           | **방치함** (그냥 두 개 다 생김 -> 에러 주범)        |
| **순서(Order)**   | **보장됨** (왼쪽 데이터 순서 유지)                   | **보장 안 됨** (분산 처리 중 뒤섞임)              |
| **Semi / Anti** | 직접 구현 필요 (`isin`, `~isin`)               | **전용 옵션 존재** (`leftsemi`, `leftanti`)  |
| **Null 조인 키**   | `NaN`끼리 매칭 안 됨 (기본값)                     | `null`끼리 매칭 안 됨 (SQL 표준 준수)              |
| **작동 방식**       | 단일 메모리에서 계산                              | 네트워크 타고 데이터 이동 (**Shuffle** 발생)        |

In [1]:
from pyspark.sql import (
    Row,
    SparkSession)
import pyspark.sql.functions as F

In [2]:
spark=(
    SparkSession
    .builder
    .appName("spakr-joins")
    .master("spark://spark-master:7077")
    .getOrCreate()
)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
26/02/02 03:45:12 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
# 왼쪽 테이블
left_df = spark.createDataFrame(
    [
        (1, "Alice"),
        (2, "Bob"),
        (3, "Charlie"),
        (4, "Diana")
    ],
    ["id", "name"]
)
# 오른쪽 테이블 
right_df = spark.createDataFrame(
    [
        (1, 5000),
        (2, 6000),
        (5, 7000)
    ],
    ["id", "salary"]
)

In [6]:
left_df.show()

+---+-------+
| id|   name|
+---+-------+
|  1|  Alice|
|  2|    Bob|
|  3|Charlie|
|  4|  Diana|
+---+-------+



In [7]:
right_df.show()

+---+------+
| id|salary|
+---+------+
|  1|  5000|
|  2|  6000|
|  5|  7000|
+---+------+



In [8]:
# inner Join

In [9]:
left_df.join(right_df,on='id',how='inner').show()

+---+-----+------+
| id| name|salary|
+---+-----+------+
|  1|Alice|  5000|
|  2|  Bob|  6000|
+---+-----+------+



In [13]:
# full Join 
# pandas how='outer' / spark how='full'

In [12]:
left_df.join(right_df,on='id',how='full').show()

+---+-------+------+
| id|   name|salary|
+---+-------+------+
|  1|  Alice|  5000|
|  2|    Bob|  6000|
|  3|Charlie|  NULL|
|  4|  Diana|  NULL|
|  5|   NULL|  7000|
+---+-------+------+



In [14]:
# Left Join

In [15]:
left_df.join(right_df,on='id',how='left').show()

+---+-------+------+
| id|   name|salary|
+---+-------+------+
|  1|  Alice|  5000|
|  2|    Bob|  6000|
|  3|Charlie|  NULL|
|  4|  Diana|  NULL|
+---+-------+------+



In [16]:
left_df.join(right_df,on='id',how='right').show()

+---+-----+------+
| id| name|salary|
+---+-----+------+
|  1|Alice|  5000|
|  2|  Bob|  6000|
|  5| NULL|  7000|
+---+-----+------+



In [19]:
# left Semi 
# 오른쪽에 있는 id만 남긴다 . 단, 데이터는 오른쪽것을 가져오지 않고 왼쪽것만!

In [20]:
left_df.join(right_df,on='id',how='leftsemi').show()

+---+-----+
| id| name|
+---+-----+
|  1|Alice|
|  2|  Bob|
+---+-----+



In [21]:
# left anti
# 오른쪽에 없는 id만 남긴다. 단 데이터는 왼쪽것만!

In [22]:
left_df.join(right_df,on='id',how='leftanti').show()

+---+-------+
| id|   name|
+---+-------+
|  3|Charlie|
|  4|  Diana|
+---+-------+



In [23]:
# 중복 컬럼 방지 

In [26]:
# 안좋은 예 -> id가 2개 생긴다 

In [25]:
left_df.join(
    right_df,
    left_df.id == right_df.id,
    "inner"
).show()

+---+-----+---+------+
| id| name| id|salary|
+---+-----+---+------+
|  1|Alice|  1|  5000|
|  2|  Bob|  2|  6000|
+---+-----+---+------+



In [27]:
# 좋은에 -> 자동병합으로 id가 하나 남는다

In [28]:
left_df.join(right_df,on=['id'],how='inner').show()

+---+-----+------+
| id| name|salary|
+---+-----+------+
|  1|Alice|  5000|
|  2|  Bob|  6000|
+---+-----+------+

